<img src="statics//iguide_logo.png" width=200 height=200 />

# Community Hydrological Model support on I-GUIDE platform - WRFHydro/National Water Model

The [I-GUIDE](https://iguide.illinois.edu/) platform integrates various state-of-the-art cyberinfrastructure (CI) capabilities to support Community Hydrological Modelling. [WRFHydro](https://ral.ucar.edu/projects/wrf_hydro), serving as the code base for the [NOAA National Water Model](https://water.noaa.gov/about/nwm) (NWM), is a leading-edge, open-source community hydrometeorological and hydrologic modelling system developed by [NCAR](https://ncar.ucar.edu/). 

This notebook demostrates the setup for a typical WRFHydro model on I-GUIDE platform leveraging different tools or services through out the entire end-to-end modelling workflow. 

In [1]:
import os
import json
import time
from datetime import datetime, timedelta
import shutil

import requests

## Setup Simulation Parameters

In [2]:
# huc12 id
huc12 = "070200030503"

# Start at 00:00 (12AM)
start_datetime = datetime(2016, 1, 1)
#                          Y   M   D
# End at 00:00 (12AM)
end_datetime = datetime(2016, 1, 7)
#                        Y   M   D

# version WRFHydro codebase on github (tag/release/commit id)
wrfhydro_version = "v5.2.0"

In [3]:
params_subset_domain = {"huc12_id": huc12, 
                        "start_date": start_datetime.strftime("%m/%d/%Y"), 
                        "end_date": end_datetime.strftime("%m/%d/%Y")}
params_subset_domain

{'huc12_id': '070200030503',
 'start_date': '01/01/2016',
 'end_date': '01/07/2016'}

## Subset DOMAIN Files with GeoEDF Data Connector on CyberGIS Compute

The source of WRFHydro DOMAIN files is [CUAHSI Domain Subsetter](https://subset.cuahsi.org/) service. I-GUIDE provides a reusable [GeoEDF Data Connector](https://dl.acm.org/doi/10.1145/3311790.3396631) ([CUAHSISubsetterInput-Connector](https://github.com/I-GUIDE/cybergis-compute-cuahsisubsetterinput-connector) ) that makes requests to CUAHSI Domain Subsetter REST APIs and retrieves the domain files ready for model use. The GeoEDF Data Connector has been integrated into [CyberGIS-Compute](https://cybergis.github.io/cybergis-compute-python-sdk/reference.html) as a job that can be invoked by users from Jupyter environment and executed on supported HPC resources. The subset domain files staged remotely is ready for use by WRFHydro model on HPC, and user has the option to download the files from HPC back to Jupyter for local manipulation.

In [4]:
import cybergis_compute_client
from cybergis_compute_client import CyberGISCompute

cybergis = CyberGISCompute(url="cgjobsup.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")
cybergis.show_ui(defaultJob="CUAHSI_Subsetter_Connector", input_params=params_subset_domain)

Retain Domain Subsetter JobID for later reference

In [5]:
jobid_cuahsi_subset_domain = cybergis.job.id
jobid_cuahsi_subset_domain

'16621261594Xdrl'

## Subset FORCINGS with GeoEDF Data Processor on CyberGIS Compute

The source of WRFHydro FORCING files is [AORC](https://registry.opendata.aws/nwm-archive/) dataset hosted on AWS. I-GUIDE provides a reusable [GeoEDF Data Processor](https://dl.acm.org/doi/10.1145/3311790.3396631) ([SubsetForcingData-Processor](https://github.com/I-GUIDE/cybergis-compute-subsetaorcforcingdata-processor) ) that subsets forcing files spatially and temporally. This GeoEDF Data Processor has been integrated into [CyberGIS-Compute](https://cybergis.github.io/cybergis-compute-python-sdk/reference.html) as a job that can be invoked by users from Jupyter environment and executed on supported HPC resources. The subset forcing files staged remotely is ready for use by WRFHydro model on HPC, and user has the option to download the files from HPC back to Jupyter for local manipulation.

In [6]:
import cybergis_compute_client
from cybergis_compute_client import CyberGISCompute

cybergis = CyberGISCompute(url="cgjobsup.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")
cybergis.show_ui(defaultJob="Subset_AORC_Forcing_Data_Processor", input_params=params_subset_domain)

Retain Forcing Processor JobID for later reference

In [7]:
jobid_subset_forcing = cybergis.job.id
jobid_subset_forcing

'1662126431G0ymZ'

## Prepare Model Configurations

In [8]:
# setEnvar.sh ---> SPATIAL_SOIL=1 export HYDRO_D=1

In [9]:
# Create a "Simulation" directory

workspace = os.getcwd()

simulation_dir = os.path.join(workspace, 'Simulation')
if os.path.exists(simulation_dir):
    shutil.rmtree(simulation_dir)
os.makedirs(simulation_dir)

#List of files
os.listdir(simulation_dir)

[]

In [10]:
! wget https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/v5.2.0/trunk/NDHMS/template/setEnvar.sh
! sed -i '/export HYDRO_D=0/c\export HYDRO_D=1' ./setEnvar.sh
! sed -i '/export SPATIAL_SOIL=0/c\export SPATIAL_SOIL=1' ./setEnvar.sh
! cat ./setEnvar.sh | grep -E 'HYDRO_D|SPATIAL_SOIL'
! mv ./setEnvar.sh {simulation_dir}

--2022-09-02 13:56:23--  https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/v5.2.0/trunk/NDHMS/template/setEnvar.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 532 [text/plain]
Saving to: ‘setEnvar.sh’

setEnvar.sh         100%[===================>]     532  --.-KB/s    in 0s      

2022-09-02 13:56:23 (29.2 MB/s) - ‘setEnvar.sh’ saved [532/532]

export HYDRO_D=1
export SPATIAL_SOIL=1


In [11]:
# namelist.hrldas --> START_YEAR START_MONTH START_DAY START_HOUR START_MIN RESTART_FILENAME_REQUESTED
start_year = start_datetime.year
start_month = "{:02d}".format(start_datetime.month)
start_day = "{:02d}".format(start_datetime.day)
start_hour = "{:02d}".format(start_datetime.hour)
start_minute = "{:02d}".format(start_datetime.minute)
khour = (end_datetime - start_datetime) / timedelta(hours=1)
khour = "{}".format(int(khour))

In [12]:
! rm -rf namelist.hrldas
! wget https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/v5.2.0/trunk/NDHMS/template/NoahMP/namelist.hrldas
! sed -i  '/HRLDAS_SETUP_FILE/c\HRLDAS_SETUP_FILE = "./DOMAIN/wrfinput_d0x.nc"' ./namelist.hrldas
! sed -i  '/START_YEAR/c\START_YEAR = '"$start_year" ./namelist.hrldas
! sed -i  '/START_MONTH/c\START_MONTH = '"$start_month" ./namelist.hrldas
! sed -i  '/START_DAY/c\START_DAY = '"$start_day" ./namelist.hrldas
! sed -i  '/START_HOUR/c\START_HOUR = '"$start_hour" ./namelist.hrldas
! sed -i  '/START_MIN/c\START_MIN = '"$start_minute" ./namelist.hrldas
! sed -i  '/KHOUR =/c\KHOUR = '"$khour" ./namelist.hrldas
! sed -i  '/RESTART_FILENAME_REQUESTED/c\!RESTART_FILENAME_REQUESTED = ""' ./namelist.hrldas
! cat ./namelist.hrldas | grep -E 'HRLDAS_SETUP_FILE|START_|KHOUR'
! mv ./namelist.hrldas {simulation_dir}

--2022-09-02 13:56:25--  https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/v5.2.0/trunk/NDHMS/template/NoahMP/namelist.hrldas
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2425 (2.4K) [text/plain]
Saving to: ‘namelist.hrldas’

namelist.hrldas     100%[===================>]   2.37K  --.-KB/s    in 0s      

2022-09-02 13:56:25 (59.4 MB/s) - ‘namelist.hrldas’ saved [2425/2425]

HRLDAS_SETUP_FILE = "./DOMAIN/wrfinput_d0x.nc"
START_YEAR = 2016
START_MONTH = 01
START_DAY = 01
START_HOUR = 00
START_MIN = 00
!RESTART_FILENAME_REQUESTED = ""
KHOUR = 144
RESTART_FREQUENCY_HOURS = 24


In [13]:
# hydro.namelist  --> RESTART_FILE
! rm -rf hydro.namelist
! wget https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/v5.2.0/trunk/NDHMS/template/HYDRO/hydro.namelist
! sed -i '/GEO_STATIC_FLNM/c\GEO_STATIC_FLNM = "./DOMAIN/geo_em.d0x.nc"' ./hydro.namelist
! sed -i '/RESTART_FILE/c\!RESTART_FILE = ""' ./hydro.namelist
! sed -i  '/GW_RESTART/c\GW_RESTART = 0' ./hydro.namelist
! sed -i  '/LSMOUT_DOMAIN/c\LSMOUT_DOMAIN = 1' ./hydro.namelist
! sed -i  '/output_gw/c\output_gw = 1' ./hydro.namelist
! sed -i '/outlake/c\outlake  = 0' ./hydro.namelist
! sed -i '/output_gw/c\output_gw  = 0' ./hydro.namelist
! sed -i '/GWBASESWCRT/c\GWBASESWCRT  = 1' ./hydro.namelist
! sed -i '/output_channelBucket_influx/c\output_channelBucket_influx  = 2' ./hydro.namelist
! sed -i '/channel_option/c\channel_option  = 2' ./hydro.namelist
! sed -i '/route_link_f/c\route_link_f  = "./DOMAIN/Route_Link.nc"' ./hydro.namelist
! sed -i '/compound_channel/c\!compound_channel  = .FALSE.' ./hydro.namelist
! sed -i '/route_lake_f/c\!route_lake_f  = "./DOMAIN/LAKEPARM.nc"' ./hydro.namelist
! sed -i '/gwbasmskfil/c\!gwbasmskfil  = "./DOMAIN/GWBASINS.nc"' ./hydro.namelist
! sed -i '/UDMP_OPT/c\UDMP_OPT  = 1' ./hydro.namelist
! sed -i '/!udmap_file/c\udmap_file = "./DOMAIN/spatialweights.nc"' ./hydro.namelist
! cat hydro.namelist | grep -E "RESTART|outlake|GWBASESWCRT|route_lake_f|gwbasmskfil"
! mv ./hydro.namelist {simulation_dir}

--2022-09-02 13:56:27--  https://raw.githubusercontent.com/NCAR/wrf_hydro_nwm_public/v5.2.0/trunk/NDHMS/template/HYDRO/hydro.namelist
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11277 (11K) [text/plain]
Saving to: ‘hydro.namelist’

hydro.namelist      100%[===================>]  11.01K  --.-KB/s    in 0s      

2022-09-02 13:56:28 (90.6 MB/s) - ‘hydro.namelist’ saved [11277/11277]

!RESTART_FILE = ""
GW_RESTART = 0
outlake  = 0
!route_lake_f  = "./DOMAIN/LAKEPARM.nc"
GWBASESWCRT  = 1
!gwbasmskfil  = "./DOMAIN/GWBASINS.nc"


## Run WRFHydro Model on HPC with CyberGIS Compute

In [14]:
params_wrfhydro = {"Model_Version": wrfhydro_version,
                   "LSM_Type": "NoahMP",
                   "Forcing_Path": jobid_subset_forcing,
                   "Domain_Path": jobid_cuahsi_subset_domain,
                   "Merge_Output": "True"}
params_wrfhydro

{'Model_Version': 'v5.2.0',
 'LSM_Type': 'NoahMP',
 'Forcing_Path': '1662126431G0ymZ',
 'Domain_Path': '16621261594Xdrl',
 'Merge_Output': 'True'}

In [15]:
import cybergis_compute_client
from cybergis_compute_client import CyberGISCompute

cybergis = CyberGISCompute(url="cgjobsup.cigi.illinois.edu", isJupyter=True, protocol="HTTPS", port=443, suffix="v2")
cybergis.create_job_by_ui(defaultJob="wrfhydro-5.x", defaultDataFolder=simulation_dir,input_params=params_wrfhydro)

Download only the Outputs_Merged/CHRTOUT folder and retain WRFHydro JobID

In [16]:
jobid_wrfhydro = cybergis.job.id
output_wrfhydro = cybergis.recentDownloadPath
!ls -LR {output_wrfhydro}

/home/jovyan/globus_download_1662127002yiEWa:
Outputs  Outputs_Merged  Simulation  slurm_log

/home/jovyan/globus_download_1662127002yiEWa/Outputs:
CHRTOUT  diag_hydro  HYDRO_RST	LDASOUT  LSMOUT  RESTART  RTOUT

/home/jovyan/globus_download_1662127002yiEWa/Outputs/CHRTOUT:
201601010000.CHRTOUT_DOMAIN1  201601040100.CHRTOUT_DOMAIN1
201601010100.CHRTOUT_DOMAIN1  201601040200.CHRTOUT_DOMAIN1
201601010200.CHRTOUT_DOMAIN1  201601040300.CHRTOUT_DOMAIN1
201601010300.CHRTOUT_DOMAIN1  201601040400.CHRTOUT_DOMAIN1
201601010400.CHRTOUT_DOMAIN1  201601040500.CHRTOUT_DOMAIN1
201601010500.CHRTOUT_DOMAIN1  201601040600.CHRTOUT_DOMAIN1
201601010600.CHRTOUT_DOMAIN1  201601040700.CHRTOUT_DOMAIN1
201601010700.CHRTOUT_DOMAIN1  201601040800.CHRTOUT_DOMAIN1
201601010800.CHRTOUT_DOMAIN1  201601040900.CHRTOUT_DOMAIN1
201601010900.CHRTOUT_DOMAIN1  201601041000.CHRTOUT_DOMAIN1
201601011000.CHRTOUT_DOMAIN1  201601041100.CHRTOUT_DOMAIN1
201601011100.CHRTOUT_DOMAIN1  201601041200.CHRTOUT_DOMAIN1
201601011200.CHRTO

## Data Visualization

In [17]:
import xarray as xr

# path of the merged file of channel routing "CHRTOUT_DOMAIN1_merged.nc"
ch_file = '{}/CHRTOUT_DOMAIN1_merged.nc'.format(output_wrfhydro)
print(ch_file)
# path of the route link "Rouet_Link.nc"
routelink ='/home/jovyan/globus_download_{}/Route_Link.nc'.format(jobid_cuahsi_subset_domain)
print(routelink)
# convert rouetlink to dataframe
route_df = xr.open_dataset(routelink).to_dataframe() # convert routelink to dataframe
route_df.gages = route_df.gages.str.decode('utf-8').str.strip()
route_df # print out the routelink dataframe

# Re-name the "Route_Link.nc" variables
cols = ['order', 'link', 'gages', 'lat', 'lon', 'to', 'from']  # columns name of original routelink dataframe
route_re_df = route_df[cols].sort_values(by=['order'])      # reduce the size of routelink dataframe to include only these columns ['order', 'link', 'gages', 'lat', 'lon', 'to', 'from']

# rename the columns
route_renm_df = route_re_df.rename(index=str, columns={"order": "stream_order",
                                       "link": "comid",
                                       "from": 'upstream_comid',
                                       'to':'downstream_comid',
                                       "gages": 'usgs_gageid',
                                       "lat": 'lat-midpoint',
                                       "lon": 'lon-midpoint'})
route_renm_df.reset_index(inplace=True)
# data.drop('linkDim', axis=1, inplace=True)
print(route_renm_df)

########################################################################
########################################################################
# Load channel routing data "CHRTOUT_DOMAIN1_merged.nc"
channel_ds = xr.open_dataset(ch_file)
########################################################################
########################################################################

# Reduce the size of the dataset to only the essential variables.
reach_ds = channel_ds
reach_ds = reach_ds[['streamflow',        # Streamflow 
                     'q_lateral',         # lateral inflow through reach
                     'qSfcLatRunoff',     # runoff from terrain routing
                     'qBucket',           # flux from groundwater (gw) bucket
                     'qBtmVertRunoff',    # runoff from bottom of soil to bucket
                     'order',
                     'velocity',
                     # 'Head',
                     'elevation']]

## clean unnecessary data
reach_ds = reach_ds.reset_coords()
reach_ds = reach_ds.drop(labels=['latitude','longitude'])
reach_ds.attrs = {}

print(reach_ds.variables['streamflow'])

/home/jovyan/globus_download_1662127002yiEWa/CHRTOUT_DOMAIN1_merged.nc
/home/jovyan/globus_download_16621261594Xdrl/Route_Link.nc
   feature_id  stream_order      comid usgs_gageid  lat-midpoint  \
0           0             1    4109714                 45.003242   
1          16             1    4111332                 44.995125   
2          15             1    4111322                 44.996651   
3          14             1    4112858                 44.914249   
4          13             1    4111692                 44.896286   
5          12             1    4111686                 44.902191   
6          11             1    4111666                 44.903225   
7          10             1    4111636                 44.905617   
8           9             1    4111606                 44.911499   
9          20             1    4111616                 44.908009   
10          7             1    4111528                 44.935139   
11          6             1    4111468                

FileNotFoundError: [Errno 2] No such file or directory: b'/home/jovyan/globus_download_1662127002yiEWa/CHRTOUT_DOMAIN1_merged.nc'

In [20]:
usgs_ids = route_renm_df.usgs_gageid.unique() # USGS IDs in the routelink data
usgs_ids

array([''], dtype=object)

In [21]:
reachid= route_renm_df[route_renm_df.usgs_gageid==usgs_gageid].comid.item()
reach_ds.sel(feature_id=reachid)['streamflow'].plot(ax=ax,
                                                    label='Total Outflow ($m^3$/s)',
                                                    color='red',
                                                    linestyle='--')

NameError: name 'usgs_gageid' is not defined

In [22]:
import matplotlib.pyplot as plt
import math
# WRFHydro (modeled) streamflow
# reachid= route_renm_df[route_renm_df.usgs_gageid==usgs_gageid].comid.item()

fig, ax = plt.subplots(1, 1, figsize=(12,6), sharex='col')
reach_ds.sel(=4111322)['streamflow'].plot(ax=ax,
                                                    label='Total Outflow ($m^3$/s)',
                                                    color='red',
                                                    linestyle='--')

SyntaxError: invalid syntax (375344825.py, line 7)

In [23]:
import xarray as xr
chrtout = xr.open_mfdataset('{}/Outputs/CHRTOUT/*CHRTOUT*'.format(output_wrfhydro),
                            combine='by_coords')
chrtout

<xarray.Dataset>
Dimensions:         (time: 145, reference_time: 1, feature_id: 42)
Coordinates:
  * time            (time) datetime64[ns] 2016-01-01 ... 2016-01-07
  * reference_time  (reference_time) datetime64[ns] 2016-01-01
  * feature_id      (feature_id) int32 4111714 4111616 ... 4111606 4111636
    latitude        (feature_id) float32 dask.array<chunksize=(42,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(42,), meta=np.ndarray>
Data variables:
    crs             (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    order           (time, feature_id) int32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    elevation       (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    streamflow      (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    q_lateral       (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    velocity        (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    Head            (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    qBucket         (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
    qBtmVertRunoff  (time, feature_id) float32 dask.array<chunksize=(1, 42), meta=np.ndarray>
Attributes: (12/18)
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0
    featureType:                timeSeries
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    model_initialization_time:  2016-01-01_00:00:00
    station_dimension:          feature_id
    model_output_valid_time:    2016-01-01_00:00:00
    ...                         ...
    model_configuration:        default
    dev_OVRTSWCRT:              1
    dev_NOAH_TIMESTEP:          3600
    dev_channel_only:           0
    dev_channelBucket_only:     0
    dev:                        dev_ prefix indicates development/internal me...

In [ ]:
#!rm -rf Simulation